In [ ]:
# installing tmdb dependency
!pip install tmdbv3api

# Importing libraries

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# using tmbd website for metadata of the movies
from tmdbv3api import TMDb
import json
import requests

tmdb = TMDb()
tmdb.api_key = 'ef8ca48cd6c13b4f6428be8546a62fb4'

# Extracting features of 2018 movies from Wikipedia

In [ ]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [ ]:
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [ ]:
df

## Function to extract data of the data from TMDb

In [ ]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    movie_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json = response.json()
    if data_json['genres']:
        genre_str = " " 
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return genre_str.join(genres)
    else:
        return np.NaN

In [ ]:
# creating genres column
df['genres'] = df['Title'].map(lambda x: get_genre(str(x)))

In [ ]:
# filtering required data
df_2018 = df.loc[:, ['Title','Cast and crew','genres']]

In [ ]:
df_2018

In [ ]:
# directors
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [ ]:
df_2018['director_name']  = df_2018['Cast and crew'].map(lambda x: get_director(x))

In [ ]:
# actor 1
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [ ]:
df_2018['actor_1_name'] = df_2018['Cast and crew'].map(lambda x: get_actor1(x))

In [ ]:
# actor 2
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [ ]:
df_2018['actor_2_name'] = df_2018['Cast and crew'].map(lambda x: get_actor2(x))

In [ ]:
# actor 3
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [ ]:
df_2018['actor_3_name'] = df_2018['Cast and crew'].map(lambda x: get_actor3(x))

In [ ]:
# renaming movie name
df_2018 = df_2018.rename(columns={'Title':'movie_title'})

In [ ]:
# converting movie name into lowercase
df_2018['movie_title'] = df_2018['movie_title'].str.lower()

In [ ]:
# filtering required data
new_df18 = df_2018.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [ ]:
# replacing nan values to unknown
new_df18['actor_2_name'] = new_df18['actor_2_name'].replace(np.nan, 'unknown')
new_df18['actor_3_name'] = new_df18['actor_3_name'].replace(np.nan, 'unknown')

In [ ]:
new_df18

# Extracting features of 2019 movies from Wikipedia

In [ ]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2019"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

# concating the data above
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

# getting movie data from tmdb
df['genres'] = df['Title'].map(lambda x: get_genre(str(x)))

# filtering desired columns
df_2019 = df.loc[:, ['Title','Cast and crew','genres']]

# cast and crew data
df_2019['director_name']  = df_2019['Cast and crew'].map(lambda x: get_director(x))
df_2019['actor_1_name'] = df_2019['Cast and crew'].map(lambda x: get_actor1(x))
df_2019['actor_2_name'] = df_2019['Cast and crew'].map(lambda x: get_actor2(x))
df_2019['actor_3_name'] = df_2019['Cast and crew'].map(lambda x: get_actor3(x))

# renaming the title
df_2019 = df_2019.rename(columns={'Title':'movie_title'})
# converting title string to lowercase
df_2019['movie_title'] = df_2019['movie_title'].str.lower()

# creating new dataframe with data needed
new_df19 = df_2019.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

# handling nan values of actors
new_df19['actor_2_name'] = new_df19['actor_2_name'].replace(np.nan, 'unknown')
new_df19['actor_3_name'] = new_df19['actor_3_name'].replace(np.nan, 'unknown')

new_df19

# Merging the 2018, 2019 data to movies data till 2017

In [ ]:
my_df = pd.concat([new_df18, new_df19],ignore_index=True)

In [ ]:
old_df = pd.read_csv('../datasets/movies_upto_2017.csv')

In [ ]:
final_df = pd.concat([old_df, my_df],ignore_index=True)

In [ ]:
final_df

In [ ]:
# checking for null values
final_df.isnull().sum()

In [ ]:
# dropping null rows
final_df.dropna(how='any', inplace=True)

In [ ]:
# saving the dataset
final_df.to_csv('../datasets/movies_upto_2019.csv', index=False)